In [2]:
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf

import cv2 as cv
import h5py
import lz4.frame
import matplotlib.dates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tqdm

import sys
import os
# Add the ptdraft folder path to the sys.path list
sys.path.append('/home/ryan/projects/Solar-Irradiance-Prediction-team07')
import utils.utils

np.set_printoptions(precision=4)

In [4]:
# hdf5_path = "/project/cq-training-1/project1/data/16bit-2014.01.01.0800.h5"
hdf5_path = "/home/ryan/data/16bit-2014.01.01.0800.h5"

hdf5_offset = 32  # this would correspond to: 2010.06.01.0800 + (32)*15min = 2010.06.01.1600
with h5py.File(hdf5_path, "r") as h5_data:
    global_start_idx = h5_data.attrs["global_dataframe_start_idx"]
    global_end_idx = h5_data.attrs["global_dataframe_end_idx"]
    archive_lut_size = global_end_idx - global_start_idx
    global_start_time = datetime.datetime.strptime(
        h5_data.attrs["global_dataframe_start_time"], "%Y.%m.%d.%H%M")
    lut_timestamps = [global_start_time + idx *
                      datetime.timedelta(minutes=15) for idx in range(archive_lut_size)]

<Attributes of HDF5 object at 140329074349648>


In [4]:
target_channels = ["ch1", "ch2", "ch3", "ch4", "ch5"]
dataframe_path = "/home/ryan/data/catalog.helios.public.20100101-20160101.pkl"

In [5]:
df = pd.read_pickle(dataframe_path) if dataframe_path else None

In [11]:
print(*df.columns.to_list(), sep="\n")

ncdf_path
hdf5_8bit_path
hdf5_8bit_offset
hdf5_16bit_path
hdf5_16bit_offset
BND_DAYTIME
BND_CLEARSKY_GHI
BND_CLOUDINESS
BND_GHI
TBL_DAYTIME
TBL_CLEARSKY_GHI
TBL_CLOUDINESS
TBL_GHI
DRA_DAYTIME
DRA_CLEARSKY_GHI
DRA_CLOUDINESS
DRA_GHI
FPK_DAYTIME
FPK_CLEARSKY_GHI
FPK_CLOUDINESS
FPK_GHI
GWN_DAYTIME
GWN_CLEARSKY_GHI
GWN_CLOUDINESS
GWN_GHI
PSU_DAYTIME
PSU_CLEARSKY_GHI
PSU_CLOUDINESS
PSU_GHI
SXF_DAYTIME
SXF_CLEARSKY_GHI
SXF_CLOUDINESS
SXF_GHI


In [30]:
df2=df[df.ncdf_path != 'nan']

In [49]:
offsets = sorted(df2.hdf5_16bit_offset.unique())

In [53]:
print(offsets)
print("The len of offsets list:", len(offsets))

[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91, 92, 93, 94, 95]
the len of offsets list: 89


In [54]:
df2.head(10).iloc[:,:5]

,ncdf_path,hdf5_8bit_path,hdf5_8bit_offset,hdf5_16bit_path,hdf5_16bit_offset
iso-datetime,,,,,
2010-04-14 19:00:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,44,/project/cq-training-1/project1/data/hdf5v5_16...,44
2010-04-14 19:15:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,45,/project/cq-training-1/project1/data/hdf5v5_16...,45
2010-04-14 19:45:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,47,/project/cq-training-1/project1/data/hdf5v5_16...,47
2010-04-14 20:00:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,48,/project/cq-training-1/project1/data/hdf5v5_16...,48
2010-04-14 20:15:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,49,/project/cq-training-1/project1/data/hdf5v5_16...,49
2010-04-14 20:30:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,50,/project/cq-training-1/project1/data/hdf5v5_16...,50
2010-04-14 20:45:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,51,/project/cq-training-1/project1/data/hdf5v5_16...,51
2010-04-14 21:15:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,53,/project/cq-training-1/project1/data/hdf5v5_16...,53
2010-04-14 21:30:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,54,/project/cq-training-1/project1/data/hdf5v5_16...,54


from datatime -> file -> offset -> get images -> samples -> batches....